# A deep dive into TOB-WGS' pipeline

Chapter 1: what does `ReblockGVCF` actually do ? Let's examine sample TOB1520 (which is an outlier for some QC metrics)

In [1]:
import hail as hl;

# All datasets in TOB-WGS are using GRCh38
hl.init(default_reference='GRCh38');


Running on Apache Spark version 3.1.2
SparkUI available at http://loic-notebook.australia-southeast1-a.c.notebooks-314505.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.67-bafea6b18247
LOGGING: writing to /home/jupyter/tob-deepdive/hail-20210618-0309-0.2.67-bafea6b18247.log


Import TOB1520 GVCF and explore structure and content


In [2]:
gvcf = hl.import_vcf('gs://cpg-tob-wgs-test/gvcf/batch1/TOB1520.g.vcf.gz', min_partitions=12, force_bgz=True)
gvcf.describe()
gvcf.info.END.show()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AS_InbreedingCoeff: array<float64>, 
        AS_QD: array<float64>, 
        AS_RAW_BaseQRankSum: str, 
        AS_RAW_MQ: str, 
        AS_RAW_MQRankSum: str, 
        AS_RAW_ReadPosRankSum: str, 
        AS_SB_TABLE: str, 
        BaseQRankSum: float64, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        ExcessHet: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQRankSum: float64, 
        RAW_MQandDP: array<int32>, 
        ReadPosRankSum: float64
    }
----------------------------------------
Entry fields:
    'AD': array<int32>
    'DP': int32
    'GQ': int32
    'GT': call


2021-06-18 03:10:33 Hail: INFO: Coerced sorted dataset


,,
locus,alleles,
locus<GRCh38>,array<str>,int32
chr1:10001,"[""T"",""<NON_REF>""]",10016
chr1:10017,"[""C"",""<NON_REF>""]",10026
chr1:10027,"[""A"",""<NON_REF>""]",10027
chr1:10028,"[""C"",""<NON_REF>""]",10038
chr1:10039,"[""A"",""<NON_REF>""]",10041
chr1:10042,"[""C"",""<NON_REF>""]",10044
chr1:10045,"[""A"",""<NON_REF>""]",10045
chr1:10046,"[""C"",""<NON_REF>""]",10046


In [3]:
hl.summarize_variants(gvcf)

2021-06-18 03:11:32 Hail: INFO: Coerced sorted dataset


Number of alleles,Count
2,30075685
3,5202620
4,235909
5,26301
6,6800
7,1784
8,1258
Allele type,Count
Symbolic,35550357
SNP,4464040


Worth noting:

of 35,550,357 variants, 30,075,685 are homozygous reference blocks

Let's load the GVCF after `ReblockGVCF` had been performed

In [4]:
rb_gvcf = hl.import_vcf('gs://cpg-tob-wgs-test-tmp/joint-calling/v2/hail/batch/728b87/1/output_gvcf.g.vcf.gz', force_bgz=True, min_partitions=12)
rb_gvcf.describe();
hl.summarize_variants(rb_gvcf);

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_QUALapprox: str, 
        AS_RAW_BaseQRankSum: str, 
        AS_RAW_MQ: str, 
        AS_RAW_MQRankSum: str, 
        AS_RAW_ReadPosRankSum: str, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SB_TABLE: str, 
        AS_SOR: array<float64>, 
        AS_VarDP: str, 
        BaseQRankSum: float64, 
        DP: int32, 
        DS: bool, 
        END: int32, 
   

2021-06-18 03:15:29 Hail: INFO: Coerced sorted dataset


Number of alleles,Count
2,24367909
3,4961977
4,101551
Allele type,Count
Symbolic,29431437
SNP,4094043
Deletion,518778
Insertion,510768
Star,41489
Complex,1


These are the annotations added by ReblockGVCF

        AC, AF, AN: ???? why would we need that on a single sample ?
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>,
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>,
        AS_QD: array<float64>, 
        AS_QUALapprox: str,
        AS_ReadPosRankSum: array<float64>,
        AS_SOR: array<float64>, 
        AS_VarDP: str,
        FS: float64, 
        MQ: float64,  
        MQ_DP: int32, 
        QD: float64, 
        QUALapprox: int32, 
        RAW_GT_COUNT: array<int32>, 
        SOR: float64, 
        VarDP: int32
and the annotations removed by ReblockGVCF

        MLEAC: array<int32>, 
        MLEAF: array<float64>, 

Additionally we lost more than 5e6 variants: not only homref blocks but also SNPs, etc.
Let's have a look at what we lost:

In [5]:
lost = gvcf.anti_join_rows(rb_gvcf.rows())
hl.summarize_variants(lost)

2021-06-18 03:23:08 Hail: INFO: Coerced sorted dataset
2021-06-18 03:23:57 Hail: INFO: Coerced sorted dataset


Number of alleles,Count
2,5807788
3,380225
4,148977
5,26301
6,6800
7,1784
8,1258
Allele type,Count
Symbolic,6373133
SNP,425204


We lost mostly homref blocks, as expected, but also other kind of variants, such as 425,204 SNPs

Let's inspect some of these lost variants (more than 2 alleles means it is not a homref block)

In [6]:
lost.filter_rows(hl.len(lost.alleles) == 3).show(50,1,truncate=30)

2021-06-18 03:30:59 Hail: INFO: Coerced sorted dataset
2021-06-18 03:31:56 Hail: INFO: Coerced sorted dataset
2021-06-18 03:32:46 Hail: INFO: Coerced sorted dataset


+---------------+
| locus         |
+---------------+
| locus<GRCh38> |
+---------------+
| chr1:10109    |
| chr1:10231    |
| chr1:10441    |
| chr1:16571    |
| chr1:16688    |
| chr1:19226    |
| chr1:49291    |
| chr1:58188    |
| chr1:83965    |
| chr1:99066    |
| chr1:99088    |
| chr1:109262   |
| chr1:183662   |
| chr1:186765   |
| chr1:187204   |
| chr1:189955   |
| chr1:190667   |
| chr1:191859   |
| chr1:197628   |
| chr1:198861   |
| chr1:198872   |
| chr1:261438   |
| chr1:269963   |
| chr1:272389   |
| chr1:275513   |
| chr1:277901   |
| chr1:284892   |
| chr1:285416   |
| chr1:286145   |
| chr1:455086   |
| chr1:590492   |
| chr1:590500   |
| chr1:601557   |
| chr1:601846   |
| chr1:602196   |
| chr1:667265   |
| chr1:690012   |
| chr1:712244   |
| chr1:737292   |
| chr1:737436   |
| chr1:757165   |
| chr1:769039   |
| chr1:780612   |
| chr1:789646   |
| chr1:789804   |
| chr1:790429   |
| chr1:798940   |
| chr1:798957   |
| chr1:801702   |
| chr1:820443   |
+---------------+

+------------------------------------------------------------------------------+
| alleles                                                                      |
+------------------------------------------------------------------------------+
| array<str>                                                                   |
+------------------------------------------------------------------------------+
| ["AACCCT","A","<NON_REF>"]                                                   |
| ["CCCCTAACCCTAACCCTAAACCCTAAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCAACCCCAACC... |
| ["CCCTA","C","<NON_REF>"]                                                    |
| ["G","A","<NON_REF>"]                                                        |
| ["G","A","<NON_REF>"]                                                        |
| ["T","A","<NON_REF>"]                                                        |
| ["C","T","<NON_REF>"]                                                        |
| ["G","T","<NON_REF>"]                                                        |
| ["AAAG","A","<NON_REF>"]                                                     |
| ["TTTC","T","<NON_REF>"]                                                     |
| ["C","G","<NON_REF>"]                                                        |
| ["C","A","<NON_REF>"]                                                        |
| ["G","C","<NON_REF>"]                                                        |
| ["T","C","<NON_REF>"]                                                        |
| ["G","A","<NON_REF>"]                                                        |
| ["A","G","<NON_REF>"]                                                        |
| ["G","A","<NON_REF>"]                                                        |
| ["C","T","<NON_REF>"]                                                        |
| ["G","C","<NON_REF>"]                                                        |
| ["G","C","<NON_REF>"]                                                        |
| ["C","T","<NON_REF>"]                                                        |
| ["A","G","<NON_REF>"]                                                        |
| ["T","A","<NON_REF>"]                                                        |
| ["C","T","<NON_REF>"]                                                        |
| ["A","G","<NON_REF>"]                                                        |
| ["C","A","<NON_REF>"]                                                        |
| ["G","A","<NON_REF>"]                                                        |
| ["T","G","<NON_REF>"]                                                        |
| ["C","CAT","<NON_REF>"]                                                      |
| ["C","A","<NON_REF>"]                                                        |
| ["TA","T","<NON_REF>"]                                                       |
| ["T","C","<NON_REF>"]                                                    

In [9]:
lost = lost.annotate_rows(nonhomref=hl.agg.count_where(lost.GT.is_non_ref())>0)
lost.aggregate_rows(hl.agg.count_where(lost.nonhomref))

2021-06-18 03:42:11 Hail: INFO: Coerced sorted dataset
2021-06-18 03:43:02 Hail: INFO: Coerced sorted dataset


156380

156,380 non homref variants were lost, let's examine some of them:

In [14]:
lost.filter_rows(lost.nonhomref).show(50,1, width=10)

2021-06-18 04:05:42 Hail: INFO: Coerced sorted dataset
2021-06-18 04:06:41 Hail: INFO: Coerced sorted dataset
2021-06-18 04:07:33 Hail: INFO: Coerced sorted dataset


+--------+
| locus  |
+--------+
| loc... |
+--------+
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
| chr... |
+--------+

+--------+
| all... |
+--------+
| arr... |
+--------+
| ["T... |
| ["T... |
| ["T... |
| ["T... |
| ["A... |
| ["T... |
| ["C... |
| ["C... |
| ["A... |
| ["C... |
| ["A... |
| ["C... |
| ["C... |
| ["A... |
| ["A... |
| ["C... |
| ["A... |
| ["C... |
| ["T... |
| ["C... |
| ["C... |
| ["G... |
| ["A... |
| ["A... |
| ["C... |
| ["A... |
| ["G... |
| ["C... |
| ["T... |
| ["C... |
| ["G... |
| ["T... |
| ["G... |
| ["C... |
| ["A... |
| ["C... |
| ["T... |
| ["A... |
| ["C... |
| ["C... |
| ["C... |
| ["C... |
| ["T... |
| ["T... |
| ["T... |
| ["C... |
| ["C... |
| ["T... |
| ["A... |
| ["C... |
+--------+

+--------+
| 'TO... |
+--------+
| arr... |
+--------+
| [13... |
| [11... |
| [0,... |
| [0,... |
| [0,... |
| [2,... |
| [0,... |
| [33... |
| [12... |
| [4,... |
| [0,... |
| [17... |
| [1,... |
| [28... |
| [11... |
| [11... |
| [0,... |
| [13... |
| [36... |
| [13... |
| [0,... |
| [0,... |
| [0,... |
| [0,... |
| [0,... |
| [12... |
| [1,... |
| [0,... |
| [0,... |
| [0,... |
| [0,... |
| [5,... |
| [2,... |
| [2,... |
| [0,... |
| [19... |
| [25... |
| [0,... |
| [0,... |
| [17... |
| [22... |
| [20... |
| [1,... |
| [13... |
| [4,... |
| [4,... |
| [4,... |
| [0,... |
| [0,... |
| [0,... |
+--------+

+--------+
| 'TO... |
+--------+
|  int32 |
+--------+
|     17 |
|     22 |
|      1 |
|      1 |
|      4 |
|     13 |
|     26 |
|     48 |
|     26 |
|     12 |
|     16 |
|     23 |
|     25 |
|     30 |
|     19 |
|     19 |
|     32 |
|     27 |
|     40 |
|     21 |
|     35 |
|     17 |
|     16 |
|     32 |
|     19 |
|     24 |
|      5 |
|     16 |
|     16 |
|     44 |
|     23 |
|     16 |
|     29 |
|     19 |
|     29 |
|     42 |
|     48 |
|      4 |
|      4 |
|     30 |
|     32 |
|     42 |
|     13 |
|     13 |
|     12 |
|     12 |
|     12 |
|     11 |
|     15 |
|     15 |
+--------+

+--------+
| 'TO... |
+--------+
|  int32 |
+--------+
|     99 |
|     99 |
|      6 |
|      6 |
|     15 |
|     48 |
|     71 |
|     99 |
|     99 |
|     37 |
|     48 |
|      1 |
|     39 |
|     24 |
|     34 |
|      0 |
|     96 |
|     99 |
|     25 |
|     99 |
|     99 |
|      8 |
|      8 |
|     96 |
|     99 |
|     99 |
|     14 |
|     48 |
|     48 |
|     99 |
|     45 |
|     65 |
|     34 |
|     53 |
|     68 |
|     99 |
|     99 |
|     13 |
|     15 |
|     26 |
|      0 |
|     99 |
|     62 |
|      1 |
|     96 |
|     96 |
|     90 |
|      2 |
|     45 |
|     46 |
+--------+

+--------+
| 'TO... |
+--------+
| call   |
+--------+
| 0/1    |
| 0/2    |
| 1/1    |
| 1/1    |
| 1/1    |
| 0/1    |
| 1/2    |
| 0/1    |
| 0/2    |
| 0/1    |
| 1/1    |
| 0/2    |
| 2/2    |
| 0/2    |
| 0/2    |
| 0/1    |
| 1/1    |
| 0/1    |
| 0/1    |
| 0/1    |
| 3/3    |
| 2/2    |
| 2/2    |
| 2/2    |
| 1/2    |
| 0/2    |
| 0/1    |
| 1/1    |
| 1/1    |
| 2/2    |
| 2/2    |
| 0/1    |
| 0/1    |
| 1/1    |
| 1/1    |
| 0/2    |
| 0/2    |
| 2/2    |
| 1/1    |
| 0/1    |
| 0/1    |
| 0/2    |
| 3/4    |
| 0/1    |
| 0/1    |
| 0/1    |
| 0/1    |
| 2/2    |
| 1/1    |
| 1/1    |
+--------+

+--------+
| 'TO... |
+--------+
|  int32 |
+--------+
|     NA |
|     NA |
|     NA |
|     NA |
|     NA |
|     NA |
|     NA |
|     NA |
|     NA |
|     NA |
|     NA |
|     NA |
|     NA |
|     NA |
|     NA |
|     NA |
|     NA |
|     NA |
|     NA |
|     NA |
|     NA |
|     NA |
|

I cannot figure out why these variants were filtered out.


For example: chr1:890446 a one nucleotide insertion C -> CA with excellent GQ=99, DP=27 with 13 reads supporting the ref and 14 reads supporting the insertion and completely unambiguous genotype likelihoods (log10-likelihood ratio = 263).